In [2]:
# Which stores rank the highest in volumes, revenues, transactions, customers, etc.?

In [3]:
# Which stores rank the highest in volumes (quantity)?
a1 = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT store_id, SUM(sales_qty) as total_units_sold
    FROM CleanedData
    GROUP BY store_id
    ORDER BY total_units_sold DESC
    LIMIT 10
    """
)

# Which stores rank the highest in volumes (quantity)?
df_a1 = client.query(a1).to_dataframe()  

df_a1


,store_id,total_units_sold
0,1212,43983417
1,1050,40702229
2,1007,37965977
3,1004,37817869
4,1066,35972643
5,1021,33634193
6,1035,33423148
7,1027,30033180
8,1040,29387920
9,1188,29129309


In [4]:
# Which stores rank the highest in volumes (weight)?
a2 = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT store_id, SUM(sales_wgt) as total_weights_sold
    FROM CleanedData
    GROUP BY store_id
    ORDER BY total_weights_sold DESC
    LIMIT 10
    """
)

# Which stores rank the highest in volumes (weight)?
df_a2 = client.query(a2).to_dataframe()  

df_a2


,store_id,total_weights_sold
0,1212,3738058.14
1,1007,3109467.16
2,1050,2887113.21
3,1066,2822230.02
4,1004,2679962.55
5,1021,2470113.91
6,1035,2435590.92
7,1011,2390760.24
8,1019,2269733.63
9,1027,2203426.15


In [5]:
# Which stores rank the highest in revenue?
b = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT store_id, SUM(sales_amt) as total_sales
    FROM CleanedData
    GROUP BY store_id
    ORDER BY total_sales DESC
    LIMIT 10
    """
)

# Which stores rank the highest in revenue?
df_b = client.query(b).to_dataframe()  

df_b


,store_id,total_sales
0,1212,2.009681e+08
1,1050,1.830785e+08
2,1004,1.722616e+08
3,1007,1.664551e+08
4,1066,1.649147e+08
5,1021,1.569396e+08
6,1035,1.547424e+08
7,1027,1.347106e+08
8,1011,1.341409e+08
9,1040,1.319841e+08


In [8]:
# Which stores rank the highest in transactions?
c = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT store_id, COUNT(trans_id) as total_trans
    FROM CleanedData
    GROUP BY store_id
    ORDER BY total_trans DESC
    LIMIT 10
    """
)

# Which stores rank the highest in transactions?
df_c = client.query(c).to_dataframe()  

df_c


,store_id,total_trans
0,1212,36401849
1,1050,32947201
2,1007,31739296
3,1004,30794895
4,1066,28929273
5,1035,27152074
6,1021,27136825
7,1027,24694438
8,1040,24295495
9,1188,23359225


In [7]:
# Which stores rank the highest in customers?
d = (
    """
    WITH FilteredTransactions AS (
        SELECT t.trans_id, t.trans_dt, t.store_id, t.cust_id, t.prod_id,
               t.sales_amt, t.sales_qty, t.sales_wgt, p.prod_category
        FROM `machine_learning.transactions` t
        JOIN `machine_learning.products` p ON t.prod_id = p.prod_id
        WHERE t.trans_dt < '2020-03-01'
        AND t.prod_id != 20189092
        AND p.prod_category NOT IN ('Gift Cards', 'Front End Service', 'Scanning Errors',
                                    'Customer Service-Misc', 'Empties and Additionals')
    ),
    CustomerVisits AS (
        SELECT cust_id,
               COUNT(*) as visit_count,
               SUM(sales_amt) as total_spent
        FROM FilteredTransactions
        GROUP BY cust_id
        HAVING COUNT(*) >= 5 AND SUM(sales_amt) >= 100
    ),
    OverVisits AS (
        SELECT cust_id
        FROM FilteredTransactions
        GROUP BY cust_id, DATE(trans_dt)
        HAVING COUNT(DISTINCT trans_id) > 10
    ),
    ValidCustomers AS (
        SELECT cv.cust_id
        FROM CustomerVisits cv
        WHERE NOT EXISTS (
            SELECT 1
            FROM OverVisits ov
            WHERE ov.cust_id = cv.cust_id
        )
    ),
    ValidTransactions AS (
        SELECT ft.trans_id, ft.trans_dt, ft.store_id, ft.cust_id, ft.prod_id,
               ft.sales_amt, ft.sales_qty, ft.sales_wgt, ft.prod_category
        FROM FilteredTransactions ft
        INNER JOIN ValidCustomers vc ON ft.cust_id = vc.cust_id
    ),
    MultipleCustTrans AS (
        SELECT trans_id
        FROM ValidTransactions
        GROUP BY trans_id
        HAVING COUNT(DISTINCT cust_id) > 1
    ),
CleanedData AS (
    SELECT *
    FROM ValidTransactions vt
    WHERE vt.trans_id NOT IN (SELECT trans_id FROM MultipleCustTrans)
    AND (
        (vt.sales_qty = 0 AND vt.sales_wgt <> 0) OR (vt.sales_qty <> 0 AND vt.sales_wgt = 0) OR (vt.sales_qty = 1 AND vt.sales_wgt <> 0)
    )
    AND (
        (vt.prod_category NOT IN ('Coupon', 'returns') AND (vt.sales_qty > 0 OR vt.sales_wgt > 0))
        OR
        (vt.prod_category IN ('Coupon', 'returns') AND (vt.sales_qty < 0 OR vt.sales_wgt < 0))
    )
)
    SELECT store_id, COUNT(cust_id) as total_cust_cnt
    FROM CleanedData
    GROUP BY store_id
    ORDER BY total_cust_cnt DESC
    LIMIT 10
    """
)

# Which stores rank the highest in customers?
df_d = client.query(d).to_dataframe()  

df_d


,store_id,total_cust_cnt
0,1212,36401852
1,1050,32947202
2,1007,31739310
3,1004,30794895
4,1066,28929273
5,1035,27152074
6,1021,27136825
7,1027,24694438
8,1040,24295495
9,1188,23359265


### Which stores rank the highest in profits?

In [ ]:
QUERY = """
WITH product_filter AS (
    SELECT *
    FROM `machine_learning.products`
    WHERE prod_category NOT IN ("Gift Cards", "Other", "Front End Service", "Scanning Errors", "Customer Service-Misc", "Empties and Additionals")
),
valid_transactions AS (
    SELECT *
    FROM `machine_learning.transactions` a 
    join product_filter b on a.prod_id  = b.prod_id 
    WHERE trans_dt < "2020-03-01"
    AND a.prod_id IN (SELECT prod_id FROM product_filter)
        AND 
        -- Logic 1: Either sales_qty or sales_wgt is zero, but not both
        ((sales_qty = 0 AND sales_wgt <> 0) OR (sales_qty <> 0 AND sales_wgt = 0))
        AND 
        -- Logics 2 and 3 are parallel conditions
        (
            (prod_category NOT IN ("Coupon", "returns") AND (sales_qty > 0 OR sales_wgt > 0))
            OR
            (prod_category IN ("Coupon", "returns") AND (sales_qty < 0 OR sales_wgt < 0))
        )
    AND sales_amt >= 0
),
transactions_per_day AS (
    SELECT cust_id, trans_dt, COUNT(DISTINCT trans_id) AS trans_per_day
    FROM valid_transactions
    GROUP BY cust_id, trans_dt
    HAVING trans_per_day <= 10
),
eligible_custs AS (
    SELECT v.cust_id
    FROM valid_transactions v
    JOIN transactions_per_day tpd ON v.cust_id = tpd.cust_id AND v.trans_dt = tpd.trans_dt
    GROUP BY v.cust_id
    HAVING COUNT(DISTINCT v.trans_id) >= 5
    AND COUNT(DISTINCT v.trans_dt) >= 5
    AND COUNT(v.trans_id) <= 20000
),
sampled_custs AS (
    SELECT cust_id
    FROM eligible_custs
    WHERE MOD(ABS(FARM_FINGERPRINT(CAST(cust_id AS STRING))), 1000) < 1
)
SELECT tx.*
FROM `valid_transactions` tx
JOIN sampled_custs ON tx.cust_id = sampled_custs.cust_id
WHERE tx.trans_dt < "2020-03-01"
"""

# Execute the query
query_job = client.query(QUERY)  # API request

# Convert to DataFrame
sample_transaction = query_job.to_dataframe()  # Waits for query to finish and converts it to DataFrame


In [ ]:
QUERY = """
WITH product_filter AS (
    SELECT *
    FROM `machine_learning.products`
    WHERE prod_category NOT IN ("Gift Cards", "Other", "Front End Service", "Scanning Errors", "Customer Service-Misc", "Empties and Additionals")
),
valid_transactions AS (
    SELECT *
    FROM `machine_learning.transactions` a 
    join product_filter b on a.prod_id  = b.prod_id 
    WHERE trans_dt < "2020-03-01"
    AND a.prod_id IN (SELECT prod_id FROM product_filter)
        AND 
        -- Logic 1: Either sales_qty or sales_wgt is zero, but not both
        ((sales_qty = 0 AND sales_wgt <> 0) OR (sales_qty <> 0 AND sales_wgt = 0))
        AND 
        -- Logics 2 and 3 are parallel conditions
        (
            (prod_category NOT IN ("Coupon", "returns") AND (sales_qty > 0 OR sales_wgt > 0))
            OR
            (prod_category IN ("Coupon", "returns") AND (sales_qty < 0 OR sales_wgt < 0))
        )
    AND sales_amt >= 0
),
transactions_per_day AS (
    SELECT cust_id, trans_dt, COUNT(DISTINCT trans_id) AS trans_per_day
    FROM valid_transactions
    GROUP BY cust_id, trans_dt
    HAVING trans_per_day <= 10
),
eligible_custs AS (
    SELECT v.cust_id
    FROM valid_transactions v
    JOIN transactions_per_day tpd ON v.cust_id = tpd.cust_id AND v.trans_dt = tpd.trans_dt
    GROUP BY v.cust_id
    HAVING COUNT(DISTINCT v.trans_id) >= 5
    AND COUNT(DISTINCT v.trans_dt) >= 5
    AND COUNT(v.trans_id) <= 20000
),
sampled_custs AS (
    SELECT cust_id
    FROM eligible_custs
    WHERE MOD(ABS(FARM_FINGERPRINT(CAST(cust_id AS STRING))), 200) < 1
)
SELECT tx.*
FROM `valid_transactions` tx
JOIN sampled_custs ON tx.cust_id = sampled_custs.cust_id
WHERE tx.trans_dt < "2020-03-01"
"""

# Execute the query
query_job = client.query(QUERY)  # API request

# Convert to DataFrame
sample_transaction = query_job.to_dataframe()  # Waits for query to finish and converts it to DataFrame


In [ ]:
# Load product and profit margin data
product_profit_margin_df = pd.read_excel("C:/Users/ctlan/OneDrive/desktop/AI at Scale/HW/Product Category Profit Margin.xlsx")

# Merge with product_profit_margin to get profit margins
merged_df = pd.merge(sample_transaction, product_profit_margin_df, on='prod_category', how='left')

# Calculate profit for each transaction
merged_df['profit'] = merged_df['sales_amt'] * merged_df['profit_margin']